In [9]:
!pip install pulp

  Obtaining dependency information for pulp from https://files.pythonhosted.org/packages/09/d7/57e71e11108203039c895643368c0d1a99fe719a6a80184edf240c33d25f/PuLP-2.8.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB 222.6 kB/s eta 0:01:20
   ---------------------------------------- 0.0/17.7 MB 222.6 kB/s eta 0:01:20
   ---------------------------------------- 0.0/17.7 MB 187.9 kB/s eta 0:01:34
   ---------------------------------------- 0.1/17.7 MB 273.8 kB/s eta 0:01:05
   ---------------------------------------- 0.1/17.7 MB 350.1 kB/s eta 0:00:51
   ---------------------------------------- 0.1/17.7 MB 400.9 kB/s eta 0:00:44
   ---------------------------------------- 0.2/17.7 MB 562.0 kB/s eta 0:00:32
    --------------------------------------- 0.2/17.7 MB 628.1 kB/s eta 0:00:28
    -------------------------------

In [15]:
import tkinter as tk
from tkinter import simpledialog, messagebox
import pulp

class TravelOptimizer:
    def __init__(self, root):
        self.root = root
        self.root.title("旅遊規劃小精靈")

        self.destinations = {}
        self.attractions_info = {}

        # 基本信息輸入部分
        self.budget_label = tk.Label(root, text="預算 (TWD):")
        self.budget_label.grid(row=0, column=0, padx=10, pady=10)
        self.budget_entry = tk.Entry(root)
        self.budget_entry.grid(row=0, column=1, padx=10, pady=10)

        self.days_label = tk.Label(root, text="旅行天數:")
        self.days_label.grid(row=1, column=0, padx=10, pady=10)
        self.days_entry = tk.Entry(root)
        self.days_entry.grid(row=1, column=1, padx=10, pady=10)

        self.daily_time_label = tk.Label(root, text="每日可用時間 (小時):")
        self.daily_time_label.grid(row=2, column=0, padx=10, pady=10)
        self.daily_time_entry = tk.Entry(root)
        self.daily_time_entry.grid(row=2, column=1, padx=10, pady=10)

        # 添加景點按鈕
        self.add_destination_button = tk.Button(root, text="添加景點", command=self.add_destination)
        self.add_destination_button.grid(row=3, column=0, columnspan=2, pady=10)

        # 增加旅行天數按鈕
        self.add_day_button = tk.Button(root, text="增加旅行天數", command=self.add_day)
        self.add_day_button.grid(row=3, column=2, columnspan=2, pady=10)

        # 計算最佳化計劃按鈕
        self.calculate_button = tk.Button(root, text="計算最佳化計劃", command=self.optimize)
        self.calculate_button.grid(row=4, column=0, columnspan=2, pady=10)

        # 重置按鈕
        self.reset_button = tk.Button(root, text="重置所有", command=self.reset_all)
        self.reset_button.grid(row=4, column=2, columnspan=2, pady=10)

        # 顯示結果文本框
        self.result_text = tk.Text(root, height=20, width=60)
        self.result_text.grid(row=5, column=0, columnspan=3, padx=10, pady=10)

        # 創建可滾動的框架
        self.attractions_canvas = tk.Canvas(root, height=300, width=600)
        self.attractions_canvas.grid(row=6, column=0, columnspan=3, padx=10, pady=10)

        # 垂直滾動條
        self.v_scrollbar = tk.Scrollbar(root, orient="vertical", command=self.attractions_canvas.yview)
        self.v_scrollbar.grid(row=6, column=3, sticky="ns")

        # 水平滾動條
        self.h_scrollbar = tk.Scrollbar(root, orient="horizontal", command=self.attractions_canvas.xview)
        self.h_scrollbar.grid(row=7, column=0, columnspan=3, sticky="ew")

        self.attractions_frame = tk.Frame(self.attractions_canvas)
        self.attractions_canvas.create_window((0, 0), window=self.attractions_frame, anchor="nw")

        self.attractions_canvas.configure(yscrollcommand=self.v_scrollbar.set, xscrollcommand=self.h_scrollbar.set)
        self.attractions_frame.bind("<Configure>", self.on_frame_configure)

        # 綁定滾輪
        self.attractions_canvas.bind_all("<MouseWheel>", self._on_mousewheel)

        # 水平滾動
        self.attractions_canvas.bind_all("<Shift-MouseWheel>", self._on_shift_mousewheel)

        self.distance_entries = {}
        self.cost_entries = {}

    def on_frame_configure(self, event):
        self.attractions_canvas.configure(scrollregion=self.attractions_canvas.bbox("all"))

    def _on_mousewheel(self, event):
        self.attractions_canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")

    def _on_shift_mousewheel(self, event):
        self.attractions_canvas.xview_scroll(int(-1 * (event.delta / 120)), "units")

    def refresh_attractions_display(self):
        for widget in self.attractions_frame.winfo_children():
            widget.destroy()

        row_index = 0
        for day in range(int(self.days_entry.get())):
            day_label = tk.Label(self.attractions_frame, text=f"第 {day + 1} 天")
            day_label.grid(row=row_index, column=0, sticky="w")
            row_index += 1

            for city, attractions in self.destinations.items():
                for attraction in attractions:
                    if self.attractions_info[attraction]["day"] == day:
                        info = self.attractions_info[attraction]
                        attraction_label = tk.Label(self.attractions_frame, text=f"{city}: {attraction} (時間: {info['time']} 小時, 花費: {info['cost']} TWD)")
                        attraction_label.grid(row=row_index, column=1, sticky="w")

                        edit_button = tk.Button(self.attractions_frame, text="修改", command=lambda a=attraction: self.edit_attraction(a))
                        edit_button.grid(row=row_index, column=2, padx=5)

                        delete_button = tk.Button(self.attractions_frame, text="刪除", command=lambda a=attraction: self.delete_attraction(a))
                        delete_button.grid(row=row_index, column=3, padx=5)

                        row_index += 1

    def is_valid_number(self, entry):
        try:
            float(entry)
            return True
        except ValueError:
            return False

    def add_day(self):
        try:
            current_days = int(self.days_entry.get())
            self.days_entry.delete(0, tk.END)
            self.days_entry.insert(0, str(current_days + 1))
            self.add_destination(day=current_days)
        except ValueError:
            messagebox.showerror("錯誤", "請輸入有效的旅行天數")

    def add_destination(self, day=None):
        total_days = int(self.days_entry.get())
        if day is None:
            for day in range(total_days):
                self.add_day_destinations(day)
        else:
            self.add_day_destinations(day)

        self.refresh_attractions_display()

    def add_day_destinations(self, day):
        messagebox.showinfo("日程安排", f"安排第 {day + 1} 天的行程")
        city = simpledialog.askstring("輸入", f"第 {day + 1} 天的城市名稱:")
        if city:
            if city not in self.destinations:
                self.destinations[city] = []

            while True:
                input_dialog = AttractionInputDialog(self.root, city)
                if not input_dialog.result:
                    break

                attraction, time, cost = input_dialog.result
                if not attraction or time is None or cost is None:
                    messagebox.showerror("錯誤", "請輸入所有字段")
                    continue

                self.destinations[city].append(attraction)
                self.attractions_info[attraction] = {"time": time, "cost": cost, "day": day, "city": city}

                add_more = messagebox.askyesno("添加更多景點", f"是否繼續添加{city}的景點?")
                if not add_more:
                    break

        self.refresh_attractions_display()

    def edit_attraction(self, attraction):
        if attraction in self.attractions_info:
            info = self.attractions_info[attraction]
            input_dialog = AttractionInputDialog(self.root, info["city"], attraction, info["time"], info["cost"])
            if input_dialog.result:
                _, time, cost = input_dialog.result
                if time is not None and cost is not None:
                    self.attractions_info[attraction]["time"] = time
                    self.attractions_info[attraction]["cost"] = cost
                    self.refresh_attractions_display()

    def delete_attraction(self, attraction):
        if attraction in self.attractions_info:
            city = self.attractions_info[attraction]["city"]
            self.destinations[city].remove(attraction)
            del self.attractions_info[attraction]
            self.refresh_attractions_display()

    def optimize(self):
        try:
            budget = float(self.budget_entry.get())
            days = int(self.days_entry.get())
            daily_time = float(self.daily_time_entry.get())
        except ValueError:
            messagebox.showerror("錯誤", "請確保預算、旅行天數和每日可用時間都是有效的數字")
            return

        # 創建問題
        problem = pulp.LpProblem("Travel Plan Optimization", pulp.LpMaximize)

        # 創建決策變數
        attraction_vars = pulp.LpVariable.dicts("Attraction", self.attractions_info.keys(), cat="Binary")

        # 添加目標函數
        problem += pulp.lpSum(attraction_vars[attraction] for attraction in self.attractions_info)

        # 添加預算限制
        problem += pulp.lpSum(attraction_vars[attraction] * self.attractions_info[attraction]["cost"] for attraction in self.attractions_info) <= budget

        # 添加每日時間限制
        for day in range(days):
            problem += pulp.lpSum(attraction_vars[attraction] * self.attractions_info[attraction]["time"]
                                  for attraction in self.attractions_info if self.attractions_info[attraction]["day"] == day) <= daily_time

        # 解決問題
        problem.solve()

        # 顯示結果
        self.result_text.delete(1.0, tk.END)
        self.result_text.insert(tk.END, "最佳旅行計劃:\n")
        total_cost = 0
        daily_plan = {day: [] for day in range(days)}
        for attraction, var in attraction_vars.items():
            if var.value() == 1:
                day = self.attractions_info[attraction]["day"]
                city = self.attractions_info[attraction]["city"]
                daily_plan[day].append(f"{city}: {attraction}")
                total_cost += self.attractions_info[attraction]["cost"]

        for day, attractions in daily_plan.items():
            self.result_text.insert(tk.END, f"\n第 {day + 1} 天:\n")
            for attraction in attractions:
                self.result_text.insert(tk.END, f"  - {attraction}\n")

        self.result_text.insert(tk.END, f"\n總花費: {total_cost} TWD\n")
        
        # 添加刷新景點顯示的邏輯
        self.refresh_attractions_display()

    def reset_all(self):
        self.budget_entry.delete(0, tk.END)
        self.days_entry.delete(0, tk.END)
        self.daily_time_entry.delete(0, tk.END)
        self.destinations.clear()
        self.attractions_info.clear()
        self.result_text.delete(1.0, tk.END)
        self.refresh_attractions_display()

class AttractionInputDialog(simpledialog.Dialog):
    def __init__(self, parent, city, attraction="", time="", cost=""):
        self.city = city
        self.attraction = attraction
        self.time = time
        self.cost = cost
        self.result = None
        super().__init__(parent, title="輸入景點資訊")

    def body(self, master):
        tk.Label(master, text=f"城市: {self.city}").grid(row=0, column=0, columnspan=2)

        tk.Label(master, text="景點名稱:").grid(row=1, column=0)
        self.attraction_entry = tk.Entry(master)
        self.attraction_entry.grid(row=1, column=1)
        self.attraction_entry.insert(0, self.attraction)

        tk.Label(master, text="參觀時間 (小時):").grid(row=2, column=0)
        self.time_entry = tk.Entry(master)
        self.time_entry.grid(row=2, column=1)
        self.time_entry.insert(0, self.time)

        tk.Label(master, text="花費 (TWD):").grid(row=3, column=0)
        self.cost_entry = tk.Entry(master)
        self.cost_entry.grid(row=3, column=1)
        self.cost_entry.insert(0, self.cost)

        return self.attraction_entry

    def apply(self):
        self.attraction = self.attraction_entry.get()
        self.time = self.time_entry.get()
        self.cost = self.cost_entry.get()

        if not self.attraction or not self.time or not self.cost:
            messagebox.showerror("錯誤", "請輸入所有字段")
            self.result = None|
        else:
            try:
                self.time = float(self.time)
                self.cost = float(self.cost)
                self.result = (self.attraction, self.time, self.cost)
            except ValueError:
                messagebox.showerror("錯誤", "請輸入有效的數字")
                self.result = None

if __name__ == "__main__":
    root = tk.Tk()
    app = TravelOptimizer(root)
    root.mainloop()


SyntaxError: invalid syntax (3138885720.py, line 276)